In [1]:
import pandas as pd
import numpy as np

### Filepaths

In [2]:
review_json_path = 'yelp_academic_dataset_review.json'
business_json_path = 'yelp_academic_dataset_business.json'
checkin_json_path = 'yelp_academic_dataset_review.json'

### Reading Business Data

In [3]:
df_b = pd.read_json(business_json_path, lines=True)

In [4]:
df_b.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."


In [5]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


## Cleaning

### Dropping Irrelevant Columns

In [6]:
df_b = df_b.drop(columns = ['postal_code', 'address', 'latitude', 'longitude', 'hours'])

### Filtering on Category = Restaurants

In [7]:
df_b = df_b.loc[df_b.categories.str.contains("Restaurants", na=False)]

In [8]:
df_b = df_b.loc[df_b['city'] == 'Las Vegas']

In [9]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6931 entries, 36 to 209372
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   6931 non-null   object 
 1   name          6931 non-null   object 
 2   city          6931 non-null   object 
 3   state         6931 non-null   object 
 4   stars         6931 non-null   float64
 5   review_count  6931 non-null   int64  
 6   is_open       6931 non-null   int64  
 7   attributes    6851 non-null   object 
 8   categories    6931 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 541.5+ KB


### NA Values

In [10]:
df_b.isna().sum()

business_id      0
name             0
city             0
state            0
stars            0
review_count     0
is_open          0
attributes      80
categories       0
dtype: int64

In [11]:
df_b = df_b[df_b['attributes'].notna()]

In [12]:
df_b.head(1)

,business_id,name,city,state,stars,review_count,is_open,attributes,categories
36,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,Las Vegas,NV,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...","Mexican, Restaurants, Fast Food"


In [13]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6851 entries, 36 to 209372
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   6851 non-null   object 
 1   name          6851 non-null   object 
 2   city          6851 non-null   object 
 3   state         6851 non-null   object 
 4   stars         6851 non-null   float64
 5   review_count  6851 non-null   int64  
 6   is_open       6851 non-null   int64  
 7   attributes    6851 non-null   object 
 8   categories    6851 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 535.2+ KB


### Reading Review Data

In [14]:
reviews_json = 'yelp_academic_dataset_review.json'
size = 500000
reviews = pd.read_json(reviews_json, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'useful':int,'funny':int,'cool':int,
                             'text':str},
                      chunksize=size)

In [15]:
chunk_list = []
for chunk_review in reviews:
    # Drop columns that aren't needed
      ##chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df_b, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df_b = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

93378 out of 500,000 related reviews
90030 out of 500,000 related reviews
94107 out of 500,000 related reviews
90089 out of 500,000 related reviews
85891 out of 500,000 related reviews
102245 out of 500,000 related reviews
88658 out of 500,000 related reviews
88173 out of 500,000 related reviews
99172 out of 500,000 related reviews
88571 out of 500,000 related reviews
91923 out of 500,000 related reviews
103631 out of 500,000 related reviews
95000 out of 500,000 related reviews
90476 out of 500,000 related reviews
97007 out of 500,000 related reviews
101127 out of 500,000 related reviews
3287 out of 500,000 related reviews


## Cleaned Data

In [16]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502765 entries, 0 to 1502764
Data columns (total 17 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   business_id   1502765 non-null  object        
 1   name          1502765 non-null  object        
 2   city          1502765 non-null  object        
 3   state         1502765 non-null  object        
 4   stars         1502765 non-null  float64       
 5   review_count  1502765 non-null  int64         
 6   is_open       1502765 non-null  int64         
 7   attributes    1502765 non-null  object        
 8   categories    1502765 non-null  object        
 9   review_id     1502765 non-null  object        
 10  user_id       1502765 non-null  object        
 11  review_stars  1502765 non-null  int64         
 12  useful        1502765 non-null  int64         
 13  funny         1502765 non-null  int64         
 14  cool          1502765 non-null  int64         
 15

In [17]:
df_b.head(3)

,business_id,name,city,state,stars,review_count,is_open,attributes,categories,review_id,user_id,review_stars,useful,funny,cool,text,date
0,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,Las Vegas,NV,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...","Mexican, Restaurants, Fast Food",WUAt5-krh075Ie44np8Hew,YDkC5VVT8s9NlIZtAr8NUA,5,0,0,0,So why would I be giving a Fast Food chain loc...,2010-11-27 09:33:34
1,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,Las Vegas,NV,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...","Mexican, Restaurants, Fast Food",S9vIs-je49jqbhpWSQ8yHg,mxtalrN7VHVQwGjv8ln5yg,2,0,0,0,I come here bout 3x's a mo. & I just can't do ...,2016-03-19 09:03:09
2,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,Las Vegas,NV,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...","Mexican, Restaurants, Fast Food",i2Soid_X8XsSTyIrPOst6g,xammeA3ftpFoAN13v5Blyg,1,2,0,0,WORST experience EVER!!!!! never have i ate an...,2015-04-02 06:14:51


In [18]:
jap = df_b.loc[df_b.categories.str.contains("Japanese", na=False)]

In [19]:
jap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148224 entries, 301 to 1502716
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   business_id   148224 non-null  object        
 1   name          148224 non-null  object        
 2   city          148224 non-null  object        
 3   state         148224 non-null  object        
 4   stars         148224 non-null  float64       
 5   review_count  148224 non-null  int64         
 6   is_open       148224 non-null  int64         
 7   attributes    148224 non-null  object        
 8   categories    148224 non-null  object        
 9   review_id     148224 non-null  object        
 10  user_id       148224 non-null  object        
 11  review_stars  148224 non-null  int64         
 12  useful        148224 non-null  int64         
 13  funny         148224 non-null  int64         
 14  cool          148224 non-null  int64         
 15  text          

In [20]:
jap['name'].unique()

array(['Veggie House', 'Bonsai Japanese Steak & Sushi',
       'Yummy Grill & Sushi', 'Oaka Sushi House', 'Mumfresh Asian Eats',
       'Tatsujin X', 'Sojo Ramen', 'Cafe Sanuki', 'Sakana', 'Teriyaki',
       'Super Sushi', 'Kan Pai Sushi', 'Anime Ramen & Teriyaki',
       'Yagyu Yakiniku', 'The SLS Buffet', 'Sweet Poké', 'Cafe de Japon',
       'BeeGee Kitchen', 'Zenshin Asian Restaurant', 'Sake Rok',
       'Teriyaki Madness', 'Taiga Modern Japanese & Thai Restaurant',
       'Zen Kitchen', 'Hachi', 'Hanabi Sushi and Rolls', 'Gogo Poke LV',
       'KUMI by Chef Akira Back', 'Tiger Sushi', 'Su Casa',
       'Ramen Arashi', 'Chop Chop Wok', 'Gangnam Street', 'Yum Yum BBQ',
       "Brian's Beach Sushi", 'Tokyo Peking Asian Restaurants',
       'Sushi 21', 'Konbea Belt Sushi', 'Kazuki Sushi', '3pm Cafe',
       'Spa Aquae', 'Sushi Mon', 'Ninja Teriyaki - Sushi 2 Go',
       'Ohjah Japanese Steakhouse', 'Shabu Grill', 'Ginseng 3',
       'Izakaya Cocokala', 'Body Sushi Catering', 'Sushi + 

## Export DF 

In [21]:
df_b.to_json('cleaned_data.json')

In [22]:
jap.to_json('jap.json')